This exercise shows how to use Monte Carlo to approximate integration, $F = \int_{0}^{1} sin (x) dx$ and $F = \int_{2}^{4} sin (x) dx$

In [ ]:
import numpy as np
import chaospy as cp
from matplotlib.pyplot import *

np.random.seed(11)
# declare function to integrate via Monte Carlo sampling
func = lambda x: np.sin(x)

# declare vector with number of samples
N = [10, 100, 1000, 10000, 100000, 1000000, 10000000] #, 10000000, 100000000

I_hat       = np.zeros(len(N))
I           = np.cos(2) - np.cos(4)
est_std_dev = np.zeros(len(N))
rms = np.zeros(len(N))

log_dec = np.zeros(len(N))
sqrt_dec = np.zeros(len(N))
linear_dec = np.zeros(len(N))
quad_dec = np.zeros(len(N))

# define new bounds
a = 2
b = 4

zeroone_sampling = True
# for each N, perform Monte Carlo integration
for i, n in enumerate(N):

    if zeroone_sampling:
        # draw uniform samples in [0, 1]
        distr_w = cp.Uniform(0, 1)
        samples_01  = distr_w.sample(size=n)
        # transform to [a, b]
        samples     = a + (b-a)*samples_01
        # approximate the integral as the expectation of the underlying function w.r.t. the uniform distribution
        # write the sample in [a, b] in terms of the generated sampled in [0, 1]
        I_hat[i]    = (b-a)*np.mean(func(samples))
        # approximate the standard deviation of the estimation
        est_std_dev[i]  = np.sqrt(np.var(func(samples), ddof=1))/np.sqrt(n)
    else:
        distr_w = cp.Uniform(2, 4)
        samples     = distr_w.sample(size=n)
        I_hat[i]    = (b-a)*np.mean(func(samples))
        est_std_dev[i] = np.sqrt( np.var(func(samples), ddof=1))/np.sqrt(n)

    rms[i] =  np.abs(I_hat[i] - I)

    if i == 0:
        log_dec[0] = est_std_dev[0]
        sqrt_dec[0] = est_std_dev[0]
        linear_dec[0] = est_std_dev[0]
        quad_dec[0] = est_std_dev[0]
    else:
        log_dec[i] = log_dec[i-1] / np.log(10)
        sqrt_dec[i] = sqrt_dec[i-1] / np.sqrt(10)
        linear_dec[i] = linear_dec[i-1] / 10
        quad_dec[i] = quad_dec[i-1] / (10**2)

    print (I, I_hat[i], rms[i], est_std_dev[i])

# plot results
loglog(N, est_std_dev, 'r', label='standard error decay')
loglog(N, rms, 'b', label=r'$\epsilon$')
loglog(N, log_dec, 'k--', label=r'log')
loglog(N, sqrt_dec, 'k-*', label=r'sqrt')
#loglog(N, linear_dec, 'k-.', label=r'linear')
#loglog(N, quad_dec, 'k-', label=r'linear')
legend(loc='best')
show()